In [4]:
# Step 0: Install Required Tools and Libraries
# Update package list
!apt-get update

# Install build tools and SRA Toolkit
!apt-get install -y build-essential sra-toolkit unzip

# Download and compile STAR
!wget https://github.com/alexdobin/STAR/archive/2.7.10a.zip
!unzip 2.7.10a.zip
!ls STAR-2.7.10a/ # List contents after unzip
!cd STAR-2.7.10a/source && make STAR
!ls STAR-2.7.10a/ # List contents after make

Hit:1 https://cli.github.com/packages stable InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:6 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [2,125 kB]
Get:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Get:10 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,826 kB]
Hit:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:12 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Hit:13 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubunt

In [5]:
# Step 1: Download FASTQ Files from SRA
# Download and convert SRR11548957 to paired-end FASTQ files
!prefetch SRR15509125
!fasterq-dump SRR15509125 --split-files
!echo $? # Check the exit status of the fasterq-dump command
!ls # List files after fasterq-dump to see if they were created


2025-11-11T07:09:59 prefetch.2.11.3: Current preference is set to retrieve SRA Normalized Format files with full base quality scores.
2025-11-11T07:09:59 prefetch.2.11.3: 1) Downloading 'SRR15509125'...
2025-11-11T07:09:59 prefetch.2.11.3: SRA Normalized Format file is being retrieved, if this is different from your preference, it may be due to current file availability.
2025-11-11T07:09:59 prefetch.2.11.3:  Downloading via HTTPS...
2025-11-11T07:10:02 prefetch.2.11.3:  HTTPS download succeed
2025-11-11T07:10:03 prefetch.2.11.3:  'SRR15509125' is valid
2025-11-11T07:10:03 prefetch.2.11.3: 1) 'SRR15509125' was downloaded successfully
2025-11-11T07:10:03 prefetch.2.11.3: 'SRR15509125' has 0 unresolved dependencies
spots read      : 407,248
reads read      : 814,496
reads written   : 814,496
0
2.7.10a.zip  SRR15509125	  SRR15509125_2.fastq  star_output
sample_data  SRR15509125_1.fastq  STAR-2.7.10a


In [6]:
# Step 2: Prepare Reference Genome (chr22)
# Create reference directory and download chr22 from Ensembl
!mkdir -p reference
!wget -P reference ftp://ftp.ensembl.org/pub/release-109/fasta/homo_sapiens/dna/Homo_sapiens.GRCh38.dna.chromosome.22.fa.gz
!wget -P reference ftp://ftp.ensembl.org/pub/release-109/gtf/homo_sapiens/Homo_sapiens.GRCh38.109.gtf.gz

# Unzip the FASTA and GTF files
!gunzip reference/Homo_sapiens.GRCh38.dna.chromosome.22.fa.gz
!gunzip reference/Homo_sapiens.GRCh38.109.gtf.gz

--2025-11-11 07:10:38--  ftp://ftp.ensembl.org/pub/release-109/fasta/homo_sapiens/dna/Homo_sapiens.GRCh38.dna.chromosome.22.fa.gz
           => ‘reference/Homo_sapiens.GRCh38.dna.chromosome.22.fa.gz’
Resolving ftp.ensembl.org (ftp.ensembl.org)... 193.62.193.169
Connecting to ftp.ensembl.org (ftp.ensembl.org)|193.62.193.169|:21... connected.
Logging in as anonymous ... Logged in!
==> SYST ... done.    ==> PWD ... done.
==> TYPE I ... done.  ==> CWD (1) /pub/release-109/fasta/homo_sapiens/dna ... done.
==> SIZE Homo_sapiens.GRCh38.dna.chromosome.22.fa.gz ... 11389810
==> PASV ... done.    ==> RETR Homo_sapiens.GRCh38.dna.chromosome.22.fa.gz ... done.
Length: 11389810 (11M) (unauthoritative)

Homo_sapiens.GRCh38 100%[===================>]  10.86M  10.1MB/s    in 1.1s    

2025-11-11 07:10:41 (10.1 MB/s) - ‘reference/Homo_sapiens.GRCh38.dna.chromosome.22.fa.gz’ saved [11389810]

--2025-11-11 07:10:41--  ftp://ftp.ensembl.org/pub/release-109/gtf/homo_sapiens/Homo_sapiens.GRCh38.109.gtf.gz
 

In [7]:
#  Step 3: Generate STAR Genome Index
# Create index directory and build genome index
!mkdir -p star_index
!/content/STAR-2.7.10a/bin/Linux_x86_64/STAR \
    --runThreadN 4 \
    --runMode genomeGenerate \
    --genomeDir star_index \
    --genomeFastaFiles reference/Homo_sapiens.GRCh38.dna.chromosome.22.fa \
    --sjdbGTFfile reference/Homo_sapiens.GRCh38.109.gtf \
    --genomeSAindexNbases 9

	/content/STAR-2.7.10a/bin/Linux_x86_64/STAR --runThreadN 4 --runMode genomeGenerate --genomeDir star_index --genomeFastaFiles reference/Homo_sapiens.GRCh38.dna.chromosome.22.fa --sjdbGTFfile reference/Homo_sapiens.GRCh38.109.gtf --genomeSAindexNbases 9
	STAR version: 2.7.10a   compiled: 2022-01-14T18:50:00-05:00 :/home/dobin/data/STAR/STARcode/STAR.master/source
Nov 11 07:11:08 ..... started STAR run
Nov 11 07:11:08 ... starting to generate Genome files
Nov 11 07:11:10 ..... processing annotations GTF
Nov 11 07:11:23 ... starting to sort Suffix Array. This may take a long time...
Nov 11 07:11:23 ... sorting Suffix Array chunks and saving them to disk...
Nov 11 07:12:16 ... loading chunks from disk, packing SA...
Nov 11 07:12:18 ... finished generating suffix array
Nov 11 07:12:18 ... generating Suffix Array index
Nov 11 07:12:19 ... completed Suffix Array index
Nov 11 07:12:19 ..... inserting junctions into the genome indices
Nov 11 07:12:30 ... writing Genome to disk ...
Nov 11 07:12

In [8]:
# Verify that the FASTQ files were created
!ls

2.7.10a.zip  sample_data  SRR15509125_1.fastq  STAR-2.7.10a  star_output
reference    SRR15509125  SRR15509125_2.fastq  star_index


In [9]:
#  Step 4: Run STAR Alignment
# Create output directory and align reads to reference genome
!mkdir -p star_output
!/content/STAR-2.7.10a/bin/Linux_x86_64/STAR \
    --runThreadN 4 \
    --genomeDir star_index \
    --readFilesIn SRR15509125_1.fastq SRR15509125_2.fastq \
    --outFileNamePrefix star_output/SRR15509125_ \
    --outSAMtype BAM SortedByCoordinate \
    --quantMode GeneCounts

	/content/STAR-2.7.10a/bin/Linux_x86_64/STAR --runThreadN 4 --genomeDir star_index --readFilesIn SRR15509125_1.fastq SRR15509125_2.fastq --outFileNamePrefix star_output/SRR15509125_ --outSAMtype BAM SortedByCoordinate --quantMode GeneCounts
	STAR version: 2.7.10a   compiled: 2022-01-14T18:50:00-05:00 :/home/dobin/data/STAR/STARcode/STAR.master/source
Nov 11 07:12:43 ..... started STAR run
Nov 11 07:12:43 ..... loading genome
Nov 11 07:12:43 ..... started mapping
Nov 11 08:33:19 ..... finished mapping
Nov 11 08:33:20 ..... started sorting BAM
Nov 11 08:33:20 ..... finished successfully


In [13]:
#  Step 5: Inspect Output Files
# List key output files
!ls -lh star_output/SRR15509125_Aligned.sortedByCoord.out.bam
!ls -lh star_output/SRR15509125_ReadsPerGene.out.tab
!cat star_output/SRR15509125_Log.final.out


-rw-r--r-- 1 root root 1.3K Nov 11 08:33 star_output/SRR15509125_Aligned.sortedByCoord.out.bam
-rw-r--r-- 1 root root 32K Nov 11 08:33 star_output/SRR15509125_ReadsPerGene.out.tab
                                 Started job on |	Nov 11 07:12:43
                             Started mapping on |	Nov 11 07:12:43
                                    Finished on |	Nov 11 08:33:20
       Mapping speed, Million of reads per hour |	0.30

                          Number of input reads |	407248
                      Average input read length |	458
                                    UNIQUE READS:
                   Uniquely mapped reads number |	7
                        Uniquely mapped reads % |	0.00%
                          Average mapped length |	64.29
                       Number of splices: Total |	18
            Number of splices: Annotated (sjdb) |	0
                       Number of splices: GT/AG |	14
                       Number of splices: GC/AG |	0
                       Number o

In [17]:
# Step 6: Download Output Files (Colab)
from google.colab import files
files.download('star_output/SRR15509125_Aligned.sortedByCoord.out.bam')
files.download('star_output/SRR15509125_ReadsPerGene.out.tab')
files.download('star_output/SRR15509125_Log.final.out')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [14]:
import plotly.graph_objects as go

# Parse the STAR log file to extract mapping statistics
log_file = 'star_output/SRR15509125_Log.final.out'
mapping_stats = {}
with open(log_file, 'r') as f:
    for line in f:
        if '%' in line:
            parts = line.strip().split('|')
            metric = parts[0].strip()
            value = float(parts[1].strip().replace('%', ''))
            mapping_stats[metric] = value

# Define categories and values for plotting
categories = [
    'Uniquely mapped reads %',
    '% of reads mapped to multiple loci',
    '% of reads mapped to too many loci',
    '% of reads unmapped: too short',
    '% of reads unmapped: other',
    '% of chimeric reads'
]
values = [mapping_stats.get(cat, 0) for cat in categories]

# Create a Plotly bar chart
fig = go.Figure(data=[go.Bar(x=categories, y=values)])

# Update layout for better readability
fig.update_layout(
    title='STAR Alignment Summary',
    xaxis_title='Read Categories',
    yaxis_title='Percentage (%)'
)

# Display the plot
fig.show()

The Plotly bar chart visualizes the STAR alignment summary based on the output from the `SRR15509125_Log.final.out` file. Each bar represents a different category of reads after alignment:

*   **Uniquely mapped reads %**: This bar shows the percentage of reads that aligned to only one unique location in the reference genome. A high percentage here indicates a successful and specific alignment.
*   **% of reads mapped to multiple loci**: This bar represents the percentage of reads that aligned to more than one location in the reference genome. These reads are often discarded or handled separately in downstream analysis.
*   **% of reads mapped to too many loci**: This indicates the percentage of reads that aligned to an excessive number of locations, often suggesting repetitive sequences or low-quality reads.
*   **% of reads unmapped: too short**: This bar shows the percentage of reads that did not align because they were too short to be confidently mapped to the reference genome.
*   **% of reads unmapped: other**: This represents the percentage of reads that failed to align for reasons other than being too short or having too many mismatches. This could include issues with read quality, adapter contamination, or reads originating from sequences not present in the reference genome.
*   **% of chimeric reads**: This bar shows the percentage of reads that aligned as chimeric, meaning they mapped to two different locations in the genome. This can indicate structural variations or gene fusions.

Analyzing these percentages helps assess the quality of the sequencing data and the efficiency of the alignment process. In this specific plot, you can observe the proportion of reads that successfully mapped uniquely, as well as the reasons for reads not mapping.

Here is a summary of the steps we have completed in this notebook:

1.  **Setup and Installation**: We started by installing the necessary tools and libraries, including `build-essential`, `sra-toolkit`, `unzip`, and the STAR aligner. We downloaded and compiled STAR from its source.
2.  **Data Download**: We downloaded the FASTQ files for the SRA accession SRR15509125 using `prefetch` and extracted the paired-end reads using `fasterq-dump`.
3.  **Reference Genome Preparation**: We downloaded the reference genome sequence (chromosome 22) and the corresponding GTF annotation file from Ensembl. These files were then unzipped.
4.  **STAR Genome Index Generation**: We used STAR to generate a genome index based on the downloaded reference genome and GTF file. This index is required for efficient alignment.
5.  **STAR Alignment**: We performed the alignment of the FASTQ reads to the generated genome index using STAR. The output included a sorted BAM file containing the alignments and a gene counts file.
6.  **Output Inspection**: We inspected the generated output files, including the sorted BAM file, the gene counts file, and the STAR log file, to check for successful execution and initial alignment statistics.
7.  **Alignment Summary Visualization**: We parsed the STAR log file and created a Plotly bar chart to visualize the alignment summary statistics, such as the percentage of uniquely mapped reads, multi-mapped reads, and unmapped reads.